In [ ]:
# Imports nécessaires

%matplotlib inline

# Suppression de l'affichage des messages d'avertissement
import warnings
warnings.filterwarnings('ignore')
import string
import time
import nltk
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import spacy
import pandas as pd
import nltk
import keras
from keras import layers

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn import model_selection
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from nltk.stem.snowball import FrenchStemmer
from keras.layers import TextVectorization
from keras.layers import Embedding
from spacy.lang.fr import French 


# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1) 
# Pour la visualisation des pipelines sklearn
set_config(display='diagram') 

Définition de fonction utile par la suite:
text_stats permet de compter le nombres de phrases dans synopsis et sera utilisé dans l'entrainement et
split_into_tokens_steam permet de séparé le texte en token et fait la désuffixation

In [ ]:
stemmer = FrenchStemmer()
def text_stats(synopsis):
    return [{"length": len(text), "num_sentences": text.count(".")} 
            for text in synopsis]
# Split les différent token et réalise la désuffixation
def split_into_tokens_steam(text):
    steam = []
    L = text.split()
    for word in L:
        steam.append(stemmer.stem(word))
    return steam

In [ ]:
#on lit le fichier csv
film_df = pd.read_csv("allocine_genres_train.csv", sep=",")
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Les colonnes contenant les informations utilisées pour l'apprentissage
X = film_df[['titre', 'synopsis']]
# La colonne contenant l'information à prédire
y = film_df.genre


In [ ]:
# Liste des mots vides de NLTK pour français + signes de ponctuation
nltk_stopwords = stopwords.words('french')+list(string.punctuation)
#on définit les différents objets qui vont être utilisé lors de l'apprentissage
desc_vectorizer_classique_synopsis = TfidfVectorizer(tokenizer = lambda x : str.split(x, ' '), stop_words=nltk_stopwords,lowercase=True, min_df=0.01,sublinear_tf=False, use_idf=False, smooth_idf=False)
desc_vectorizer_classique_titre = TfidfVectorizer(tokenizer = lambda x : str.split(x, ' '), stop_words=nltk_stopwords,lowercase=True,sublinear_tf=False, use_idf=False, smooth_idf=False)
desc_vectorizer_steam_synopsis = TfidfVectorizer(tokenizer = split_into_tokens_steam, stop_words=nltk_stopwords,lowercase=True, min_df=0.01,sublinear_tf=False, use_idf=False, smooth_idf=False)
desc_vectorizer_steam_titre = TfidfVectorizer(tokenizer = split_into_tokens_steam, stop_words=nltk_stopwords,lowercase=True,sublinear_tf=False, use_idf=False, smooth_idf=False)


text_stats_transformer = FunctionTransformer(text_stats)
text_stats_vectorizer = DictVectorizer(sparse=False)
min_max_scaler = MinMaxScaler()

# Modèles à comparer
models = [
    ('Baseline', DummyClassifier(strategy='most_frequent')),
    ('Mutinomial NB', MultinomialNB()),
    ('CART', DecisionTreeClassifier()),
    ('LR', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('Random forest', RandomForestClassifier())
]


On utilise la vectorisation sans désuffixation

In [ ]:

column_trans = ColumnTransformer(
     [
         ('titre_tfidf', desc_vectorizer_classique_titre, 'titre'),
         # Colonne 'synopsis' : tf-idf
         ('synopsis_tfidf', desc_vectorizer_classique_synopsis, 'synopsis'),
         # Colonne 'synopsis' : statistiques
         (
             'synopsis_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ), 
             'synopsis'
         )
     ],
)
# Evaluation de chaque résultat l'un après l'autre
scores = []
names = []
scoring = 'macro F1'
# Validation croisée à 5 plis
kfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
# Itération sur les modèles
for name, model in models:
    # Ajout du nom du modèle à la liste name
    names.append(name)
    # Création de la pipeline pour le modèle
    model_pipeline = make_pipeline(column_trans, model)
    # Validation croisée
    y_pred = model_selection.cross_val_predict(model_pipeline, 
                                               X, y, 
                                               cv=kfold)
    print(name)
    print(classification_report(y, y_pred))
    f1 = metrics.f1_score(y, y_pred, average='macro')
    scores.append(f1)

# Représentation graphique des résultats
indices = np.arange(len(scores))
fig = plt.figure()
plt.barh(indices, scores, .2, label="score", color='b')
plt.yticks(())
for i, c in zip(indices, names):
    plt.text(-.3, i, c)
plt.show()

On utilise des vecteurs avec désuffixation

In [ ]:

column_trans = ColumnTransformer(
     [
         ('titre_tfidf', desc_vectorizer_steam_titre, 'titre'),
         # Colonne 'synopsis' : tf-idf
         ('synopsis_tfidf', desc_vectorizer_steam_synopsis, 'synopsis'),
         # Colonne 'synopsis' : statistiques
         (
             'synopsis_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ), 
             'synopsis'
         )
     ],
)
# Evaluation de chaque résultat l'un après l'autre
scores = []
names = []
scoring = 'macro F1'
# Validation croisée à 5 plis
kfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
# Itération sur les modèles
for name, model in models:
    # Ajout du nom du modèle à la liste name
    names.append(name)
    # Création de la pipeline pour le modèle
    model_pipeline = make_pipeline(column_trans, model)
    # Validation croisée
    y_pred = model_selection.cross_val_predict(model_pipeline, 
                                               X, y, 
                                               cv=kfold)
    print(name)
    print(classification_report(y, y_pred))
    f1 = metrics.f1_score(y, y_pred, average='macro')
    scores.append(f1)

# Représentation graphique des résultats
indices = np.arange(len(scores))
fig = plt.figure()
plt.barh(indices, scores, .2, label="score", color='b')
plt.yticks(())
for i, c in zip(indices, names):
    plt.text(-.3, i, c)
plt.show()